# Introduction to dimensionality reduction

## Setup
### Imports

In [ ]:
import pandas as pd                                     # for dataset manipulation (DataFrames)
import numpy as np                                      # allows some mathematical operations
import matplotlib.pyplot as plt                         # library used to display graphs
import seaborn as sns                                   # more convenient visualisation library for dataframes
import missingno as msno                                # utility library for missing values
from sklearn.model_selection import train_test_split    # for classification
from sklearn.neighbors import KNeighborsClassifier      # for classification
from sklearn.decomposition import PCA                   # for Principal Component Analysis
import time                                             # for execution time measurement

### Loading the dataset

In [ ]:
fives = np.loadtxt("fives.txt", delimiter=",")
sixes = np.loadtxt("sixes.txt", delimiter=",")

# for practical reasons, we convert these arrays to a pandas dataframe
df_fives = pd.DataFrame(fives)
df_sixes = pd.DataFrame(sixes)

# we also create a dataframe containing all numbers for later classification
df_fives["number"] = 5 # data labeling
df_sixes["number"] = 6 # data labeling
df = pd.concat([df_fives, df_sixes], ignore_index=True)
df_fives.drop("number", inplace=True,axis=1)
df_sixes.drop("number", inplace=True,axis=1)

### Utility functions

In [ ]:
def show_image(row, title=""):
    """This function takes a row of 256 pixels and displays it as a greyscale image"""
    image = np.reshape(row, (16, 16))
    plt.imshow(image, cmap="gray")
    plt.title(title)
    plt.show()

def split_data(data):
    X = data.drop("number", axis=1)
    y = data.number
    return train_test_split(X, y,
                            test_size=0.33,  # 10% of the data will be used for testing
                            random_state=42,  # ensures reproducibility of the test
                            stratify=y  # ensures the proportion of each class
                            )

def print_knn_score(scores, data_type=""):
    max_score = max(scores)
    k_values_max_score = [i + 1 for i, v in enumerate(scores) if v == max_score]
    print(f'Max {data_type} score {max_score * 100} % for k = {[i for i in k_values_max_score]}')

def prediction_knn(data):
    """ KNN-based classification. """
    X_train, X_test, y_train, y_test = split_data(data)
    test_scores = []
    train_scores = []

    for k in range(1, 15):
        knn = KNeighborsClassifier(k)
        knn.fit(X_train, y_train)
        train_scores.append(knn.score(X_train, y_train))  # "score" for KNN is the accuracy of the classification
        test_scores.append(knn.score(X_test, y_test))

    print_knn_score(train_scores, "train")
    print_knn_score(test_scores, "test")

def run_measure_time(function, **kwargs):
    start_time = time.time()
    function(**kwargs)
    print("--- %s seconds ---" % (time.time() - start_time))

### Baseline test

In [ ]:
# equivalent to prediction_knn(data=df), but also measures time using our utility function
run_measure_time(prediction_knn, data=df)

### Observing the dataset

In [ ]:
index = 0 # index of the image to show
show_image(fives[index, :])
show_image(sixes[index, :])
# to do the same from a pandas dataframe
# show_image(df_fives.iloc[0].to_numpy())

Using what you have learned in the previous lessons, examine the dataset and see what you can learn about it.

In [ ]:
# Your code here

*[Your comments here]*

## Principal Component Analysis (PCA)
### Pre-processing

When using PCA, it is necessary to standardize the data. Pixel values are already between -1 and 1, so we only need to center the data.
Using what you have learned last time, center the data in `df_fives` and `df_sixes` respectively.

*Hint: You can refer to the data preparation practical and use `sklearn`'s `StandardScaler`*.

⚠️Centering means removing the mean value. You do not need to divide by the standard deviation. `sklearn`'s `StandardScaler`*, check its documentation to see how.
⚠️Store the standardized data in new variables, because we will need the original data later.

In fact, this step is not strictly necessary here, because we will use `sklearn`'s implementation of PCA, which already includes standardization. However, it is important to keep in mind the importance of this step (and to know how to do it yourself).

In [ ]:
# Your code here
df5_std = ...
df6_std = ...

### Understanding what PCA does
In this part, we will try to visualize what PCA does. For this, we will start with an "average" image, and progressively add PCA components. For this visualization step, we will only be using the dataframe containing the handwritten fives.

#### Step 1: Compute the "average 5"
Create a vector (pandas Series, for example) that contains the "mean 5".
Display this vector as an image. How do you interpret this ?

In [ ]:
# Your code here

#### Step 2: Obtain the components from PCA
Read the code cell below. Using the documentation, explain what is done at each step.

In [ ]:
pca = PCA()                              # your comment here
pca.fit(df_fives)                        # ...
components5 = pca.components_            # ...
projection5 = pca.transform(df_fives)    # ...

#### Step 3: Reconstruct an image progressively
In this part, we will reconstruct an image from its PCA components.

**Questions**:
- Understand and explain the line marked with a question mark ❓.
- Observe the resulting images. How many components are necessary to obtain a "nice" image? How do you interpret this?
- By modifying the code below, display a reconstruction with 10 components. What do you observe?
- By modifying the code below, try displaying other instances of the number five from the dataset.

In [ ]:
image_index = 3
original_image = df_fives.iloc[image_index].to_numpy()
show_image(original_image, "Original image")      # we first display the original image

reconstructed_image = df_fives.mean().to_numpy()
show_image(reconstructed_image, "Mean image") # then we display the mean image

for i in range(0,3): # and finally we reconstruct the image using the components
    reconstructed_image = reconstructed_image + projection5[image_index,i] * components5[i] # ❓
    show_image(reconstructed_image, f"Using {i+1} component{'s' if i>0 else ''}")

#### Bonus step: Doing the same with the number 6
Now that you understand how to use `scikit-learn`'s PCA, make the same observations with the database of number 6's.

In [ ]:
# Your code here

*[Your comments here]*

### Using PCA for classification
A possible use for dimensionality reduction is to help machine learning algorithms.
In the code cell below, we use PCA for the entire dataset (numbers 5 and 6) and store the projection coefficients in the `projection` variable.
`df_projection` contains the same data as a `pandas` dataframe.

In [ ]:
df_unlabeled = df.drop("number", inplace=False, axis=1)
pca = PCA()
pca.fit(df_unlabeled)
projection = pca.transform(df_unlabeled)

df_projection = pd.DataFrame(projection)
df_projection["number"] = df["number"] # label the data for visualization and classification

#### Questions
- Using the code in the cell below, display a two-dimensional `scatterplot` with a different color for 5's and 6's. Which dimensions should you use?
- What can you observe? How do you think this can help machine learning algorithms?
- Try changing the features you display. How does the `scatterplot` change? How do you interpret this?
- Try performing a prediction on the two-dimensional dataset. Compare the results with the baseline test on the full dataset, both in terms of accuracy and computational time. How do you interpret this?

In [ ]:
dimension1 = 0    # should be an integer indicating which PCA component to pick
dimension2 = 1    # should be an integer indicating which PCA component to pick
df_two_dimensional = df_projection[[dimension1, dimension2, "number"]]

# Your code here

*[Your comments here]*